In [ ]:
#*********************************
# Multi-process Matrix Multiplication
#
# Manuel Serna-Aguilera
# Spring 2021
#*********************************
import multiprocessing as mp
from multiprocessing import Pool, TimeoutError
import numpy as np
import time
import os

In [ ]:
#=================================
# Serial version of recursive matrix multiplication
'''
Input:
    C: output (numpy) array
    A: first input (numpy) array
    B: second input (numpy) array
'''
#=================================
def p_matrix_mult_recursive_serial(C, A, B):
    n = A.shape[0] # get number of rows

    if n == 1:
        return np.dot(A, B) # single-element matrices
    else:
        # let T be a new n X n matrix
        T = np.zeros((n,n))

        # Partition A,B,C, and T into n/2 X n/2 submatrices
        mid = int(n/2)

        A11 = A[0:mid, 0:mid]
        A12 = A[0:mid, mid:n]
        A21 = A[mid:n, 0:mid]
        A22 = A[mid:n, mid:n]

        B11 = B[0:mid, 0:mid]
        B12 = B[0:mid, mid:n]
        B21 = B[mid:n, 0:mid]
        B22 = B[mid:n, mid:n]

        C11 = C[0:mid, 0:mid]
        C12 = C[0:mid, mid:n]
        C21 = C[mid:n, 0:mid]
        C22 = C[mid:n, mid:n]

        T11 = T[0:mid, 0:mid]
        T12 = T[0:mid, mid:n]
        T21 = T[mid:n, 0:mid]
        T22 = T[mid:n, mid:n]

        # Call recursive cases
        C11 = p_matrix_mult_recursive(C11, A11, B11)
        C12 = p_matrix_mult_recursive(C12, A11, B12)
        C21 = p_matrix_mult_recursive(C21, A21, B11)
        C22 = p_matrix_mult_recursive(C22, A21, B12)

        T11 = p_matrix_mult_recursive(T11, A12, B21)
        T12 = p_matrix_mult_recursive(T12, A12, B22)
        T21 = p_matrix_mult_recursive(T21, A22, B21)
        T22 = p_matrix_mult_recursive(T22, A22, B22)

        # Copy values back to C and T
        C[0:mid, 0:mid] = C11
        C[0:mid, mid:n] = C12
        C[mid:n, 0:mid] = C21
        C[mid:n, mid:n] = C22

        T[0:mid, 0:mid] = T11
        T[0:mid, mid:n] = T12
        T[mid:n, 0:mid] = T21
        T[mid:n, mid:n] = T22

        # Parallel loop to get elements in C (n X n)
        for i in range(n):
            for j in range(n):
                C[i, j] = C[i, j] + T[i, j]

        return C

In [ ]:
#=================================
# Multiprocess version of recursive add (analogous to parallel for loop)
'''
Input:
    C: (numpy array) matrix whose contents will be updated
    T: (numpy array) matrix whose elements will be added to corresponding indices in C
Return:
    NA

NOTE: matrices assumed to be n X n, where n is a power of 2
'''
#=================================
def add_multiprocess(C, T):
    n = C.shape[0]

    if n == 1:
        C = C + T 
    else:
        # Partition C and T into n/2 X n/2 submatrices
        mid = int(n/2)

        # Call recursive cases as multiple async processes (once we get out of 'with' block, everything is synced)
        with Pool(processes=4) as pool:
            pool.apply_async(add_multiprocess, (C[0:mid, 0:mid], T[0:mid, 0:mid]))
            pool.apply_async(add_multiprocess, (C[0:mid, mid:n], T[0:mid, mid:n]))
            pool.apply_async(add_multiprocess, (C[mid:n, 0:mid], T[mid:n, 0:mid]))
            pool.apply_async(add_multiprocess, (C[mid:n, mid:n], T[mid:n, mid:n]))

        # Parallel portion finished, this should prevent memory leaks and pool is closed
        pool.close()
        pool.join()

In [ ]:
#=================================
# Multiprocess version for recursive matrix multiplication
'''
Input:
    C: output (numpy) array where results are written to
    A: first input (numpy) array
    B: second input (numpy) array
Return:
    NA

NOTE: matrices assumed to be n X n, where n is a power of 2
'''
#=================================
def p_matrix_mult_recursive_multiprocess(C, A, B):
    n = A.shape[0] # get number of rows

    if n == 1:
        C = np.dot(A, B) # single-element matrices
    else:
        # Let T be a new n X n matrix
        T = np.zeros((n,n))

        # Partition A,B,C, and T into n/2 X n/2 submatrices
        mid = int(n/2)

        # Call recursive cases as multiple async processes (once we get out of 'with' block, everything is synced)
        with Pool(processes=8) as pool:
            pool.apply_async(p_matrix_mult_recursive_multiprocess, (C[0:mid, 0:mid], A[0:mid, 0:mid], B[0:mid, 0:mid]))
            pool.apply_async(p_matrix_mult_recursive_multiprocess, (C[0:mid, mid:n], A[0:mid, 0:mid], B[0:mid, mid:n]))
            pool.apply_async(p_matrix_mult_recursive_multiprocess, (C[mid:n, 0:mid], A[mid:n, 0:mid], B[0:mid, 0:mid]))
            pool.apply_async(p_matrix_mult_recursive_multiprocess, (C[mid:n, mid:n], A[mid:n, 0:mid], B[0:mid, mid:n]))
            pool.apply_async(p_matrix_mult_recursive_multiprocess, (T[0:mid, 0:mid], A[0:mid, mid:n], B[mid:n, 0:mid]))
            pool.apply_async(p_matrix_mult_recursive_multiprocess, (T[0:mid, mid:n], A[0:mid, mid:n], B[mid:n, mid:n]))
            pool.apply_async(p_matrix_mult_recursive_multiprocess, (T[mid:n, 0:mid], A[mid:n, mid:n], B[mid:n, 0:mid]))
            pool.apply_async(p_matrix_mult_recursive_multiprocess, (T[mid:n, mid:n], A[mid:n, mid:n], B[mid:n, mid:n]))

        # Parallel portion finished, this should prevent memory leaks and pool is closed
        pool.close()
        pool.join()

        # Add
        add_multiprocess(C, T)

In [ ]:
# Init 
rng = np.random.default_rng()

In [ ]:
# Test: setup
TESTS = 100 # tests to run for particular size-n matrix
n = 1 # initial size of matrices
#max_n = 32 # MAX size of matrices
#max_n = 64
#max_n = 128
#max_n = 256
#max_n = 1024
#max_n = 2048
max_n = 4096
scale = 2 # increase size of matrices by this factor

times = np.zeros((int(np.log2(max_n)+1))) # record average times here for each size n

In [ ]:
# Run tests on increasing matrix sizes
index=0
while n <= max_n:
    print('n = {}'.format(n))

    for test in range(TESTS):
        matrixA = rng.standard_normal(size=(n,n), dtype=np.float32)
        matrixA.flags.writeable = False # input matrix 'A' read-only
        matrixB = rng.standard_normal(size=(n,n), dtype=np.float32)
        matrixB.flags.writeable = False # input matrix 'B' read-only

        matrixC = np.zeros((n,n))

        start = time.time()
        p_matrix_mult_recursive_multiprocess(C=matrixC, A=matrixA, B=matrixB)
        finish = time.time() - start
        #print('{}) {}'.format(index, finish))

        times[index] += finish

    # Update
    times[index] = times[index] / TESTS
    n *= scale
    index += 1

n = 1
n = 2
n = 4
n = 8
n = 16
n = 32
n = 64
n = 128
n = 256
n = 512
n = 1024
n = 2048
n = 4096


In [ ]:
# Print average timings for each matrix size
print("======================================")
print('Average times for each matrix size')

print('size n)\ttime (seconds)')
for i in range(times.size):
    print('2^{})\t{:.8f}'.format(i, times[i]))

Average times for each matrix size
size n)	time (seconds)
2^0)	0.00000194
2^1)	0.29108859
2^2)	0.29529122
2^3)	0.29903442
2^4)	0.30430891
2^5)	0.31532744
2^6)	0.32314299
2^7)	0.32713509
2^8)	0.33292394
2^9)	0.34292706
2^10)	0.35500550
2^11)	0.39274744
2^12)	0.47496295
